<a href="https://colab.research.google.com/github/maxigaarp/Gestion-De-Datos-en-R/blob/main/Proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Proyecto Grupal 

## Entrega: Domingo 29 de Agosto 2021

Este trabajo consistirá en armar una base de datos en SQL y un informe descriptivo de los datos y la investigación. El objetivo del proyecto es predecir asistencia a clases en base a los datos meteorologicos de precipitación.

La base de datos debe tener la informacion de 3 diferentes fuentes de información:

 * Asistencia Declarada Mensual ([MINEDUC](http://datos.mineduc.cl/dashboards/19844/asistencia-declarada-mensual-por-ano/)) años: (2015-2019)
  Acá se incluyen los porcentajes de asistencia y la cantidad de dias trabajados de todos los alumnos en los distintos colegios de Chile 
 * Reportes de precipitacion Mensual ([CR2](https://www.cr2.cl/camels-cl/)): Datos historicos de precipitacion las distintas estaciones meteorologicas en Chile con localizacion geografica.
 * Localizacion Colegios (Propia/Google Maps):
 Datos de la ubicacion geografica de los colegios en Chile.

Para esto generar esta base de datos se espera que se incluyan a lo menos 3 diferentes tablas:
 * alumnos: Informacion relevante de los alumnos. A lo menos incluir 3 atributos.
 * colegios: Informacion relevante de los colegios. A lo menos incluir 3 atributos, ademas:  nombre (o ID) de la estacion meteorologica más cercana y la distancia a esta, asumiendo latitud y longitud como coordenadas cartesianas.
 * estudia_en: Informacion relevante del vinculo entre el colegio, mes y alumnos. En la tabla se debe incorporar la informacion mensual para cada alumno en terminos de: asistencia, precipitacion. En este caso la precipitacion debe ser una interpolacion en base a los datos de las precipitaciones de las estaciones meteorologicas y la geolocalizacion de los colegios.

Cabe destacar que las bases de datos no se encuentran libres de errores por lo que para consolidar la informacion hay que solucionar problemas de consistencia, completitud, validez. Entre otros se busca que:

 * Que los datos para cada uno atributos se encuentren en el formato adecuado y unico. Para lograr esto revisar los atributos más importantes: asistencia y precipitacion.
 * Una unica forma de expresar valores nulos (unificar distintos tipos de descripcion de valores nulos, ejemplo: " ", NA, 0, -9999, etc). 
 * Seleccionar una cantidad tolerable (mayor que 2 al menos) de valores nulos para las series temporales (los registros de las estaciones meteorologicas). Interpolar los valores nulos en las series.

Cabe destacar que las tablas deben unificarse en SQL cargando las tablas mediante RSQL. Luego de esto se espera que se seleccione y utilice un algoritmo adecuado para generar una predicción de asistencia a clases en base a la precipitacion mensual, puede ser alguno de los siguientes: k-means, regresion lineal, regresion logistica. Para esta ultima parte, dado el tamaño de los datos de entrenamiento, puede ser que no puedan cargar en R la base de datos, si esto les pasa tienen 4 opciones:

 * Cargar y utilizar regresion lineal en trozos de la base de datos mediante los comandos dbSendQuery y dbFetch: Para cada trozo utilizar el metodo de prediccion adecuado y luego promediar los pesos de cada uno para obtener un predictor final (es decir obtener un solo conjunto de parametros beta).
 * Usar un metodo de muestreo en la base de datos: Para ello deben señalar cual metodo ocupan y verificar que sea adecuado, por ejemplo: que incluya alumnos de todas las regiones de chile.
 * Usar package speedGLM para usar regresion en BigData: Estos algoritmos estan especializados para grandes volumenes de datos. Se encuentra adjunto un pequeño codigo para ayudarlos a utilizar esta herramienta
 * Analizar un subconjunto de los datos: para ello pueden seleccionar alguna caracteristica que les acomode, por ejemplo la comuna y utilizar regresion lineal dentro de ese conjunto de datos. Sin embargo deberán analizar las diferencias entre distintos grupos, por ejemplo: si se subdivide por la region analizar las diferencias (los parametros son mas relevantes, el R² aumenta, etc...) entre regiones del norte, centro y sur de chile. 

La entrega del proyecto debe tener dos partes: 
* El codigo donde se genera la base de datos y el desarrollo del algoritmo de prediccion. Puede ser Colab o script de R, debe tener comentarios solo con el objetivo de guiar al ayudante en la correccion.
* Un documento de no mas de 3 paginas (sin incluir gráficos) con la documentacion de los datos (diagrama ER, tipo de datos, descripcion, dominio, etc. Pueden guiarse por el esquema de registro en las bases de datos del MINEDUC.) y además un resumen de lo desarrollado en el proyecto (como se generaron las variables, como se procesaron y que resultados se obtuvieron)

Para una mejor calidad en el informe usar graficos y tablas que pueden incluir en anexos.



Datos disponibles en: https://drive.google.com/drive/folders/17Or8k6rhYvkaeEn_pD10za8A9jluYfUF?usp=sharing

#Descargar datos

In [ ]:
install.packages("RSQLite")


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘plogr’




In [ ]:
library(tidyverse)
library(data.table)
library(RSQLite)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.3     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose




##Asistencia

In [ ]:
system("gdown --id 1-q1ydcu6afA3LQ9uxlh9J9B9kvEJvrRs")

In [ ]:
unzip("/content/DatosProyecto.zip")

In [ ]:
library(stringr)

In [ ]:
lista <- list.files("/content/content/drive/MyDrive/Gestion de Datos/Datos Proyecto/Datos asistencia")
directorio <- "/content/content/drive/MyDrive/Gestion de Datos/Datos Proyecto/Datos asistencia/"

for (name in lista){
  if (str_detect(name, ".rar")){
    foldername<-substr(name, 1, nchar(name)-4)
    system(paste("mkdir 'Asistencia/",foldername,"'",sep=""))
    system(paste("unrar x '",directorio,name,"' 'Asistencia/",substr(name, 1, nchar(name)-4),"/'", sep=""),intern = T)
  }
  else {
    unzip(paste(directorio,name,sep=""), exdir="Asistencia")
  }
}


In [ ]:
lista_de_csvs <- list.files("Asistencia",pattern = ".(CSV|csv)$", recursive = TRUE)


In [ ]:
conn <- dbConnect(RSQLite::SQLite(), "mineduc.db")


## Precipitacion

In [ ]:
unzip("/content/content/drive/MyDrive/Gestion de Datos/Datos Proyecto/cr2_prAmon_2019.zip")

In [ ]:
,pp <- read.csv("/content/cr2_prAmon_2019/cr2_prAmon_2019.txt",na = "-9999", header =F)
pp <- setNames(as.data.frame(t(pp[,-1])),as.character(pp[,1]))


In [ ]:
ppp <- pp %>% select( c("codigo_estacion","nombre", "latitud","longitud") | "2015-01":"2019-12")%>%
        pivot_longer(cols = "2015-01":"2019-12",
        values_to = "Precipitacion",
        names_to = c("Año", "Mes"),
        names_pattern = "(....)-(..)")


## Geolocalización




In [ ]:
geocoles <- read.csv("/content/content/drive/MyDrive/Gestion de Datos/Datos Proyecto/colesgeo.csv", row.names=1)

In [ ]:
head(geocoles)

,RBD,NOM_RBD,NOM_COM_RBD,query,lon,lat
,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,1,LICEO POLITECNICO ARICA,ARICA,"LICEO POLITECNICO ARICA , ARICA",-70.29469,-18.48719
2,3,ESC. PEDRO VICENTE GUTIERREZ TORRES,ARICA,"ESC. PEDRO VICENTE GUTIERREZ TORRES , ARICA",-70.30278,-18.48696
3,4,LICEO OCTAVIO PALMA PEREZ,ARICA,"LICEO OCTAVIO PALMA PEREZ , ARICA",-70.30792,-18.47417
4,5,JOVINA NARANJO FERNANDEZ,ARICA,"JOVINA NARANJO FERNANDEZ , ARICA",-70.31345,-18.47680
5,7,L. POLI. ANTONIO VARAS DE LA BARRA,ARICA,"L. POLI. ANTONIO VARAS DE LA BARRA , ARICA",-70.28795,-18.47595
6,8,COLEGIO INTEGRADO EDUARDO FREI MONTALVA,ARICA,"COLEGIO INTEGRADO EDUARDO FREI MONTALVA , ARICA",-70.31737,-18.48267


#Opcion 1: subdividir y promediar

In [ ]:
q<-dbSendQuery(conn, "
  select        #Aca va la 
  *             #consulta para generar 
  from alumnos  #datos de entrenamiento
  ")

Warning message:
“Closing open result set, pending rows”


In [ ]:
dbFetch(q,1000000) #entrega trozos de la consulta q de base de datos de largo 1000000

In [ ]:
m1<-glm(ASIS_PROMEDIO~Latitud+Precipitacion, data=dbFetch(q,1000000))
summary(m1)


Call:
glm(formula = ASIS_PROMEDIO ~ Latitud + Precipitacion, data = dbFetch(conection, 
    6000))

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-0.89219  -0.02048   0.10816   0.15818   0.20659  

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    7.521e-01  2.135e-02  35.230  < 2e-16 ***
Latitud       -2.685e-03  6.304e-04  -4.259 2.08e-05 ***
Precipitacion -1.125e-04  4.734e-05  -2.377   0.0175 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 0.07488802)

    Null deviance: 450.52  on 5999  degrees of freedom
Residual deviance: 449.10  on 5997  degrees of freedom
AIC: 1481.7

Number of Fisher Scoring iterations: 2


In [ ]:
coefficients(m1)

(Intercept)       Latitud Precipitacion 
 0.7521474445 -0.0026849993 -0.0001125419

Luego promediar todos estos coeficientes para obtener un valor de los coeficientes final.

#Opcion 2: Muestreo

In [ ]:
q<-dbGetQuery(conn, "
  select        #Aca va la 
  *             #consulta para generar 
  from alumnos  #datos de entrenamiento
  order by RANDOM()
  limit 10000000
  ")

Seleccionar una muestra aleatoria de la tabla y realizar analisis. Explicar el metodo de muestreo y revisar si es acorde a la pregunta, ¿tiene tamaño suficiente para representar los datos?

#Opcion 3: SpeedGLM

SpeedGLM es un algoritmo para ajustar modelos lineares generales a grandes volumenes de datos (BigData). Se necesita crear una funcion para cargar los datos por trozos

In [ ]:
install.packages("speedglm")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(speedglm)

In [ ]:
 #funcion para entregar datos

 make.data<-function(chunksize){       
     conection<-NULL
     function(reset=FALSE){
     if(reset){
       conn <- dbConnect(RSQLite::SQLite(), "mineduc.db")
       conection<<-dbSendQuery(conn, "
                                  select        #Aca va la 
                                  *             #consulta para generar 
                                  from alumnos  #datos de entrenamiento
                                  ")
     } else{
       rval<-dbFetch(conection,chunksize)
       if ((nrow(rval)==0)) {
            conection<<-NULL
            rval<-NULL
       }
       return(rval)
     }
  }
}

In [ ]:
#para validar

da<-make.data(chunksize=50)

da(reset=T) #1: opens the connection to "data1.txt"
da(reset=F) #2: reads the first 50 rows (out of 120) of the dataset
da(reset=F) #3: reads the second 50 rows (out of 120) of the dataset
da(reset=F) #3: reads the last 20 rows (out of 120) of the dataset
da(reset=F) #4: is NULL: this latter command closes the connection




ASIS_PROMEDIO,Latitud,Precipitacion
<dbl>,<dbl>,<dbl>
0.7500,-33.44102,7.4
0.6667,-33.44102,0.0
0.5294,-33.44102,2.0
0.7500,-33.44102,0.0
0.9091,-33.44102,38.3
0.8095,-33.44102,111.4
0.7000,-33.44102,23.7
0.5238,-33.44102,27.7
0.8095,-33.44102,4.5


ASIS_PROMEDIO,Latitud,Precipitacion
<dbl>,<dbl>,<dbl>
1.0000,-18.49699,0
1.0000,-18.49699,0
1.0000,-18.49699,0
0.9500,-18.49699,0
1.0000,-18.49699,0
1.0000,-18.49699,0
1.0000,-18.49699,0
0.9500,-18.49699,0
1.0000,-18.49699,0


ASIS_PROMEDIO,Latitud,Precipitacion
<dbl>,<dbl>,<dbl>
0.9474000,-18.4462,0
1.0000000,-18.4462,0
0.8636000,-18.4462,0
0.6842000,-18.4462,0
1.0000000,-18.4462,0
0.7619000,-18.4462,0
0.6842000,-18.4462,0
0.8889000,-18.4462,0
0.8571000,-18.4462,0


NULL

In [ ]:
#entrenar con trozos de 10000000 datos
da<-make.data(chunksize=10000000)

b1<-shglm(ASIS_PROMEDIO~Latitud+Precipitacion,datafun=da)
summary(b1) 


Generalized Linear Model of class 'speedglm':

Call:  shglm(formula = ASIS_PROMEDIO ~ Latitud + Precipitacion, datafun = da) 

Coefficients:
 ------------------------------------------------------------------ 
                Estimate Std. Error t value  Pr(>|t|)    
(Intercept)    8.033e-01  1.244e-04  6456.7  0.00e+00 ***
Latitud       -1.101e-03  3.623e-06  -303.8  0.00e+00 ***
Precipitacion  1.958e-06  6.750e-08    29.0 5.83e-185 ***

------------------------------------------------------------------- 
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 

---
null df: 151925055; null deviance: 10842672;
residuals df: 151925053; residuals deviance: 10835912;
# obs.: 151925056; # non-zero weighted obs.: 151925056;
AIC: 446513069; log Likelihood: -223256530;
RSS: 10835912; dispersion: 0.07132406; iterations: 3;
rank: 3; max tolerance: 0e+00; convergence: TRUE.